In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
# import gc


In [2]:
stock_data = pd.read_csv('TOP_100_Stock_Name.csv', encoding='latin-1' , header=None)

type_of_data = 'dividend-yield-history'
urls = []
for i in range(1, len(stock_data)):
    urls.append('https://www.macrotrends.net/stocks/charts/' + str(stock_data[1][i]) + '/' + str(stock_data[0][i]) + '/' + str(type_of_data))


In [3]:
def parse_page(driver):
    driver.execute_script("window.scrollTo(0 , 1000)")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # g tag with this XPath : //*[@id="chartdiv"]/div/div/div[2]/div[3]/div/div/svg/g[12]/g
    # Find all circle elements
    circles = soup.find_all('circle')

    # Extract and print the aria-label attributes
    aria_labels = [circle['aria-label'] for circle in circles if 'aria-label' in circle.attrs]
    print(aria_labels)
    # find all circle and get aria-labels
    # circles = transform.find_all('circle')
    data = []
    # for circle in circles:
    #     data.append(circle['aria-label'])
    #     print(circle['aria-label'])
    return data
    



In [4]:
for count, url in enumerate(urls):
    filename = f'{type_of_data}\{stock_data[1][count + 1]}_{type_of_data}_Year_Dividend_history.csv'
    # Check if the file exists
    if not os.path.exists(filename):
        try: 
            time.sleep(2)
            chrome_options = webdriver.ChromeOptions()
            chrome_options.add_argument('--ignore-certificate-errors')
            chrome_options.add_argument('--disable-web-security')
            chrome_options.add_argument('--allow-running-insecure-content')
            driver = webdriver.Chrome()
            driver.implicitly_wait(4)
            driver.get(url)
            driver.maximize_window()
            # Wait for the page to load
            element = WebDriverWait(driver, 2).until(
                EC.presence_of_element_located((By.ID, "main_content"))
            )
            
            
            try:
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "Button__StyledButton-a1qza5-0.jkvvVr"))).click()
                print('Click Accept All Button')
            except:
                print("No cookie consent pop-up found or failed to click 'Accept all'.")
        
            # Get the page source
            data = parse_page(driver)
            # Create a DataFrame
            df = pd.DataFrame(data)
            # Save the DataFrame to a CSV file
            df.to_csv(f'{type_of_data}\{stock_data[1][count + 1]}_{type_of_data}_Year_Dividend_history.csv', index=False , header=None)
            print("Save " + str(stock_data[1][count + 1]) + f" On {type_of_data}")
            # Close the browser
            driver.quit()
        except:
            print("Error On " + str(stock_data[1][count + 1]))
            driver.quit()
    else:
        print(stock_data[0][count + 1] + f"_{type_of_data}" + " already exist.")

<string>:2: SyntaxWarning: invalid escape sequence '\{'
<string>:32: SyntaxWarning: invalid escape sequence '\{'
<>:2: SyntaxWarning: invalid escape sequence '\{'
<>:32: SyntaxWarning: invalid escape sequence '\{'
<string>:2: SyntaxWarning: invalid escape sequence '\{'
<string>:32: SyntaxWarning: invalid escape sequence '\{'
<>:2: SyntaxWarning: invalid escape sequence '\{'
<>:32: SyntaxWarning: invalid escape sequence '\{'
C:\Users\Ali Mirash\AppData\Local\Temp\ipykernel_27740\1317938867.py:2: SyntaxWarning: invalid escape sequence '\{'
  filename = f'{type_of_data}\{stock_data[1][count + 1]}_{type_of_data}_Year_Dividend_history.csv'
C:\Users\Ali Mirash\AppData\Local\Temp\ipykernel_27740\1317938867.py:32: SyntaxWarning: invalid escape sequence '\{'
  df.to_csv(f'{type_of_data}\{stock_data[1][count + 1]}_{type_of_data}_Year_Dividend_history.csv', index=False , header=None)


Linde_dividend-yield-history already exist.
Click Accept All Button
[]
Save FCX On dividend-yield-history
Error On SHW
